In [1]:
import pandas as pd
import os
import json
from utility_automation import *
from utility_Sparsity import *
from utility_thresholding import *
from utility_Feature_selaction import *
import re



In [2]:
config_path = "./config.json"
def load_config():
    with open(config_path, "r") as f:
        config = json.load(f)
    return config
config = load_config()

In [3]:
# This function will config summary using given input config.

def Config_Creation():
    # load defination summary
    df_summary=pd.read_csv(config["input_Data_summary"])
     # extracting the granurality level using def summary. simply it tell what level of granurality we are using for this brand (example:[Def L1, L1, Def L2, L2,.....Def Ln, Ln])
    pattern = re.compile(r"^L\d(\sDef)?$")
    granurality_level = [col for col in df_summary.columns if pattern.match(col)]
    #Created a variable dictionary for all 6 variables with default values using config input.
    variables={"parent_level":config["default_values"]["parent_level"],
                "granularity_lavel_after_Parent_level": len(granurality_level)/2-config["default_values"]["parent_level"],
                "green_thr": 5,
                "orange_thr": 5,
                "green_active": 10,
                "orange_active": 10}
    
    
    # Find Max value of parent_level whichever media type is using.
    # initial it will take default value of parent_level and if any media type have greater than that it will get updated.
    parent_level_max=int(variables['parent_level'])
    for i in config['parent_level']:
        if config['parent_level'][i]>parent_level_max:
            parent_level_max=int(config['parent_level'][i])
            
    # Here we take columns only till parent_level_Max do make config summary   
    #here we multiple with 2 beause to define one granurality we have to give 2 values (example : Def L1, L1)
    #due to cutting till parent_level_max some rows have same values till granurality so now we will remore duplicate.
    config_summary=df_summary[granurality_level[:parent_level_max*2]] 
    config_summary=config_summary.drop_duplicates()
    config_summary=config_summary.reset_index().drop('index', axis=1)
    
    # here we assigne value for 6 variable fol all row of config_summary 
    # here it will check if any exception case (diffrent values from default value) avaibale then use that value otherwise use default values.
    
    for i in ["parent_level","granularity_lavel_after_Parent_level", "green_thr", "orange_thr","green_active",  "orange_active"]: # updating values for these Variable one by one
        for index, row in config_summary.iterrows(): 
            f=False
            for j in config[f'{i}']: #check for all available cases
                L=j.split("|") # make list of filter (assume Brand|Earn Media:3 for parent_level, then L will ["Brand", "Earn Media"])
                # blew code will ensure that variables value only get overwrite from default value only for that which the satisfied gien conditon
                is_subset = set(L).issubset(set(row.to_list()))
                if is_subset:
                    f=True
                    break
            if f:
                config_summary.loc[index, f'{i}'] = config[f'{i}'][j] # assigning Defual value
            else:
                config_summary.loc[index, f'{i}']=variables[i]
            #here we upadte the vlues for that who have diffrent value within it group
            #(Example: for TV we take till CP level but within TV for TV Social we just want till CP then this functionalty will be useful )
            if i!="parent_level": # we are not updating the parent_level as it can't have diffrent value for subgroup
                f=False
                for j in config[f'{i}']["exception_case_in_Next_level"]:
                    L=j.split("|")
                    is_subset = set(L).issubset(set(row.to_list()))
                    if is_subset:
                        f=True
                        break
                if f:
                    config_summary.loc[index, f'{i}'] = config[f'{i}']["exception_case_in_Next_level"][j]
                    
    #above we took granurality till parent_level_max but different Media have different parent_level so we take according to individual parent_level
    for index, row in config_summary.iterrows():
        parent=int(row["parent_level"])
        config_summary.loc[index, config_summary.columns[parent*2:parent_level_max*2]] = np.nan
    config_summary.drop_duplicates(inplace=True)
    return config_summary, granurality_level

In [4]:
#This function take as input Def Summary and AIO and give the Impressions and Cost according to date for all possible variables.

def Format_Conversion(config):
    #loading AIO file and Def_summary file
    df_in=pd.read_csv(config["input_AIO_file_path"])
    df_in[config["date_col"]] = pd.to_datetime(df_in[config["date_col"]])
    df_summary=pd.read_csv(config["input_Data_summary"])
    
    #here we are defining one More metric with default value is "Cost". Metric 1 is user defined but Metric 2 is fixed and auto-creted here
    df_summary["Metric 2"]="Cost"
    
    #with the help of pattern("Def L1", "L1",.... ) in Def Summary columns, we extract the granularity_level.
    # level_summary is created with thw help of granurality_lavel
    pattern = re.compile(r"^L\d(\sDef)?$")
    granurality_level = [col for col in df_summary.columns if pattern.match(col)]
    level_summary = {granurality_level[i]: granurality_level[i+1] for i in range(0, len(granurality_level), 2)}
    
    #Filter out the data from AIO file accorung to Start date and End date.
    filter_dict = {config["date_col"] :list(pd.date_range(config["date_filter"]["start_date"], config["date_filter"]["end_date"]))}
    for key in filter_dict.keys():
        df_in = df_in[df_in[key].isin(filter_dict[key])]
        
    # generate filter and target dictionaries wrt summary sheet and raw data.
    target_dict_Impression,target_dict_Cost, filter_dict = generate_filter_dict(
                df_summary, config["common_filter"], level_summary, config["target_metric"]+["Metric 2"]
            )
    # generate dict with filter tag and corresponding pivot dataframe.
    tagged_df_Impresssion,tagged_df_cost = generate_tagged_df(
                df_in,
                config["common_filter"],
                filter_dict,
                target_dict_Impression,
                target_dict_Cost,
                config['date_col'],
                config["date_format"],
                config["prefix"] ,)

    df_merge_cost = generate_merge_df(tagged_df_cost, config['date_col'], config["date_filter"]["start_date"], config["date_filter"]["end_date"], config["date_format"])
    df_merge_cost = df_merge_cost.fillna(0)
    df_merge_Impression = generate_merge_df(tagged_df_Impresssion, config['date_col'], config["date_filter"]["start_date"], config["date_filter"]["end_date"], config["date_format"])
    df_merge_Impression = df_merge_Impression.fillna(0)
    
    # df_merge_cost.to_csv("output/Cost.csv", index=False)
    # df_merge_Impression.to_csv('output/Impression.csv', index=False)
    
    return df_merge_Impression,  df_merge_cost
    

In [5]:
#This function create summary on daily level and weekly level for variables with some metrics (like %yeraly spend, %overall spend,  %yearly active, %overall active,...)

def raw_conversion(imp, cost, config_summary, granurality_level):
    imp_daily=pd.DataFrame()
    imp_weekly=pd.DataFrame()
    spend_daily=pd.DataFrame()
    for i, row in config_summary.iterrows():
        parent_level = int(row['parent_level'])
        selected = row[:int(parent_level)*2]
        value =  '|'.join(str(v) for v in selected if pd.notna(v) and str(v).strip() != "")
        
        # below code filter out those columns from input files which have same granurality till parent_level and 
        # from output the all metric will be calculated for each variables
        col=[config['date_col']]+[col for col in imp.columns if value in col]
        imp_t=imp[col]
        cost_t=cost[col]
        target_list=["Impressions/GRPs", "Spend"]
        for target in target_list:
            if target=="Impressions/GRPs":
                imp_daily_t=create_daily_data(imp_t, config['date_col'], target, granurality_level, parent_level) 
                imp_weekly_t=create_weekly_data(imp_t, config['date_col'], target, granurality_level, parent_level)
            else:
                spend_daily_t=create_daily_data(cost_t, config['date_col'], target, granurality_level, parent_level)           
        imp_daily=pd.concat([imp_daily, imp_daily_t])
        imp_weekly=pd.concat([imp_weekly, imp_weekly_t])
        spend_daily=pd.concat([spend_daily, spend_daily_t])  
           
    # imp_daily.to_excel("output/imp_daily.xlsx", index=False)
    # imp_weekly.to_excel("output/imp_weekly.xlsx", index=False)
    # spend_daily.to_excel("output/spend_daily.xlsx", index=False)
    
    #combining all metric together (from both weekly and daily)
    pivot_df=create_pivot_ready_data(spend_daily,imp_daily,imp_weekly)
    pivot_df.drop_duplicates(inplace=True)
    
    #here we are putting dummy values for empty cells so that we can use pivot table funtion in next step smoothly 
    #here dummy value can we anything but take care it shlod not be similare to operation or any media. we will remore this dummy values in next steps
    for col in granurality_level:
        pivot_df[col] = pivot_df[col].fillna("Raja_Babu")
        pivot_df[col] = pivot_df[col].astype('str').str.replace('0','None')
    # pivot_df.to_excel("output/pivot_ready_data.xlsx", index=False)
    return pivot_df
        

In [6]:
def thresholding_Conditions(filtered, merged_columns, granurality_level,granularity, summary_rows, green_thr, orange_thr, green_active, orange_active, len_granularity, parent_level):
    value_list=[
        "%Weekly Overall Active",
        "%Weekly Year Active",
        "Overall Spend%",
        "Year Spend%",
        "Overall Spend",
        "%Overall Spend Channel",
        "%Year Spend Channel"]

    if len_granularity-2*parent_level-2*granularity==0:
        pivot_index = granurality_level
        pivot_values = value_list
    else: 
        pivot_index = granurality_level[:-(len_granularity-2*parent_level-2*granularity)]
        pivot_values = granurality_level[-(len_granularity-2*parent_level-2*granularity):] + value_list
    table=create_pivot_table(filtered, pivot_index, pivot_values)
    table_cpa=create_threshold_data(table,  green_thr, orange_thr, green_active, orange_active)
    temp=create_summary(table, merged_columns, granularity, green_thr, orange_thr, green_active, orange_active)
    summary_rows.append(temp)
    granularity =1
    pivot_index = granurality_level[:-(len_granularity-2*parent_level-2*granularity)]
    pivot_values = granurality_level[-(len_granularity-2*parent_level-2*granularity):] + value_list
    table=create_pivot_table(filtered, pivot_index, pivot_values)
    table_cp=create_threshold_data(table, green_thr, orange_thr, green_active, orange_active)
    temp=create_summary(table, merged_columns, granularity, green_thr, orange_thr, green_active, orange_active)
    summary_rows.append(temp)
    return table_cpa, table_cp

In [7]:
def Variable_list_feature_list_generation(pivot_df, granurality_level, config_summary):    
    Variable_List=pd.DataFrame()
    feature_list=pd.DataFrame()
    summary_rows = []
    for i, row in config_summary.iterrows():
        parent_level = int(row['parent_level'])
        selected = row[:parent_level*2].dropna().tolist()
        cols_to_match = config_summary.columns[:len(selected)]
        mask = (pivot_df[cols_to_match] == row[cols_to_match]).all(axis=1)
        merged_columns =  '|'.join(str(v) for v in selected if pd.notna(v) and str(v).strip() != "")
        filtered = pivot_df[mask].copy()
        if filtered.shape[0] == 0:
            print("No data")
            continue
        green_thr=row["green_thr"]
        orange_thr=row["orange_thr"]
        green_active= row["green_active"]
        orange_active=row["orange_active"]
        granularity_lavel_after_Parent_level=int(row["granularity_lavel_after_Parent_level"])
        len_granularity=int(len(granurality_level))
        sheet_cpa, sheet_cp=thresholding_Conditions(filtered, merged_columns, granurality_level, granularity_lavel_after_Parent_level, summary_rows, green_thr, orange_thr, green_active, orange_active, len_granularity, parent_level)
        sheet_cpa['Colour'] = sheet_cpa[['Overall Green','Overall Orange']].apply(color,axis=1) 
        sheet_cpa=Merging_Type(sheet_cpa,sheet_cp, parent_level, orange_thr)
        Variable_List= pd.concat([Variable_List, sheet_cpa], ignore_index=True)
        #implemently audience maping and platform mapping only for paid media
        if "|Paid Media|" in merged_columns and "|Halo|" not in merged_columns and "|Master Brand|" not in merged_columns:
            sheet_cpa=Audience_Others(sheet_cpa, parent_level, granularity_lavel_after_Parent_level, len_granularity)
            sheet_cpa=Platform_Others(sheet_cpa, parent_level,granularity_lavel_after_Parent_level,len_granularity)
        feature_list=pd.concat([feature_list, sheet_cpa], ignore_index=True)
    
    #here we will remore that dummy values from variable list and feature list   
    pattern = re.compile(r"^Raja_Babu$", re.IGNORECASE)
    for col in Variable_List.select_dtypes(include=['object']).columns:
        Variable_List[col] = Variable_List[col].apply(lambda x: '' if isinstance(x, str) and pattern.match(x) else x)
    for col in feature_list.select_dtypes(include=['object']).columns:
        feature_list[col] = feature_list[col].apply(lambda x: '' if isinstance(x, str) and pattern.match(x) else x)
    feature_list=feature_list[granurality_level+['Merging type']]
    feature_list["Remarks"] = None
    return Variable_List, feature_list, feature_list.drop_duplicates() , pd.DataFrame(summary_rows)

In [8]:
config_summary,granurality_level =Config_Creation()


In [9]:
imp, cost=Format_Conversion(config)
pivot_df=raw_conversion(imp, cost, config_summary, granurality_level)

/var/folders/ps/y9g7f0_52yggnl9pvx1l76zh0000gp/T/ipykernel_12269/2346988458.py:5: DtypeWarning: Columns (14,23,28,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_in=pd.read_csv(config["input_AIO_file_path"])


Merged df using the dictionary. The final shape is (1581, 331)
Merged df using the dictionary. The final shape is (1581, 331)


In [10]:
Variable_List, feature_list_with_duplicates, feature_list,summary=Variable_list_feature_list_generation(pivot_df, granurality_level, config_summary)

In [11]:
# Create copies to avoid changing the original DataFrames
feature_list_copy = feature_list_with_duplicates.copy()
variable_list_copy = Variable_List.copy()

# Add MultiIndex column headers to the copies
feature_list_copy.columns = pd.MultiIndex.from_product([["Default Feature List"], feature_list_copy.columns])
variable_list_copy.columns = pd.MultiIndex.from_product([["Default Variable List"], variable_list_copy.columns])

# Concatenate side by side (new DataFrame)
final_df = pd.concat([feature_list_copy, variable_list_copy], axis=1)



In [14]:

# Export all to one Excel file with multiple sheets
with pd.ExcelWriter('output/Feature_List.xlsx', engine='openpyxl') as writer:
    summary.to_excel(writer, sheet_name='Summary', index=False)
    config_summary.to_excel(writer, sheet_name='Config_Summary', index=False)
    Variable_List[granurality_level+["Merging type"]].to_excel(writer, sheet_name='Variable_List', index=False)
    feature_list.to_excel(writer, sheet_name='Feature_List', index=False)
    final_df.to_excel(writer, sheet_name='Tracker_Dict', index=True)
